In [1]:
import pandas as pd
import sys
from data_reader import fes_reader, perf_reader, well_renaming, rename_columns
from tqdm import tqdm
import bisect
import json
import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                'cygwin': '\\',
                'darwin': '/',
                'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])

In [3]:
perf_path1 = replace_slash('input_data\\' + 'armitz/miner/ARX_PERF.xlsx')
fes_path1 = replace_slash('input_data\\armitz/miner/QINF.xlsx')

perf_path = replace_slash('input_data\\' + 'Перфорация.json')
fes_path = replace_slash('input_data\\РИГИС.json')

In [4]:
with open(fes_path, 'r') as f:
    data = json.dumps(json.load(f))
with open(perf_path, 'r') as f:
    data_perf = json.dumps(json.load(f))

In [5]:
perf_df = pd.read_json(data_perf, orient='records')

fes_df = pd.read_json(data, orient='records')

In [6]:
perf_df1 = pd.read_excel(perf_path1, engine='openpyxl', skiprows=1)
fes_df1 = pd.read_excel(fes_path1, engine='openpyxl', skiprows=1)

In [7]:
perf_df.rename(columns=lambda x: x.lower().strip(), inplace=True)
rename_columns(perf_df)

perf_df1.rename(columns=lambda x: x.lower().strip(), inplace=True)
rename_columns(perf_df1)

In [8]:
fes_df.rename(columns=lambda x: x.lower().strip(), inplace=True)
rename_columns(fes_df)

fes_df1.rename(columns=lambda x: x.lower().strip(), inplace=True)
rename_columns(fes_df1)

In [9]:
perf_df['well'] = perf_df['well'].apply(well_renaming)
perf_df1['well'] = perf_df1['well'].apply(well_renaming)
fes_df['well'] = fes_df['well'].apply(well_renaming)
fes_df1['well'] = fes_df1['well'].apply(well_renaming)


In [14]:
perf_ints = perf_reader(perf_path)
fes_dict = fes_reader(fes_path)
SOIL_CUT = 60

started reading perf xl
done reading perf xl and started processing perf data
started reading fes xl
done reading fes xl
done processing data


In [17]:
len(fes_dict.keys())

1574

In [13]:
len(fes_df['well'].unique())

1580

In [18]:
len(perf_ints.keys())

1579